These are my notes I made while watching the PyCon 2019 video 'Lambda Calculus from the Ground Up' (David Beazley).

-------

**Everything**, repeat EVERYTHING is a function. That too of only one argument. 

In [1]:
def f(x): # Currently we are using def just to ease in to the lambda notation.
    def g(y):
        return x(y)
    return g

In [2]:
f(lambda x: x**2)(10)

100

In [3]:
(lambda x:x**2)(10)

100

Functions can be created to act as gates. 

In [4]:
def LEFT(a):
    def f(b):
        return a
    return f

def RIGHT(a):
    def f(b):
        return b
    return f

Let's go away from def notation to lambda notation.

In [5]:
LEFT  = lambda x: lambda y: x
RIGHT = lambda x: lambda y: y

In [6]:
LEFT(42)(100)  # the arguments are supposed to be functions. But, we will cheat to explore LEFT and RIGHT.

42

In [7]:
RIGHT(42)(100)

100

## Boolean Algebra with Functions

In [8]:
def TRUE(x):
    return lambda y: x

def FALSE(x):
    return lambda y: y

def NOT(x):
    return x(FALSE)(TRUE)

Assert statement are just to understand the code better. They are not allowed in lambda calculus at the moment.

In [9]:
assert NOT(TRUE) is FALSE
assert NOT(FALSE) is TRUE

In [10]:
def AND(x):
    return lambda y: NOT(x)(x)(y) # this is what I came up with the help of truth table

In [11]:
# but below is a simpler solution equivalent
def AND(x):
    return lambda y: x(y)(x)

In [12]:
assert AND(TRUE)(TRUE)  is TRUE
assert AND(TRUE)(FALSE)  is FALSE
assert AND(FALSE)(TRUE)  is FALSE
assert AND(FALSE)(FALSE)  is FALSE

In [13]:
def OR(x):
    return lambda y: x(x)(y)

In [14]:
assert OR(TRUE)(TRUE)  is TRUE
assert OR(TRUE)(FALSE)  is TRUE
assert OR(FALSE)(TRUE)  is TRUE
assert OR(FALSE)(FALSE)  is FALSE

## Numbers. 

Remember that in Lambda Calculus, everything is a function. Numbers too are functions.

In [16]:
ZERO  = lambda f: lambda x: x
ONE   = lambda f: lambda x: f(x)
TWO   = lambda f: lambda x: f(f(x))
THREE = lambda f: lambda x: f(f(f(x)))
FOUR  = lambda f: lambda x: f(f(f(f(x))))
FIVE  = lambda f: lambda x: f(f(f(f(f(x)))))

In [17]:
# Following snippets are illegal. But they help us debug/visualize what is happening

def incr(x):
    return x+1 # illegal as per the rules, but helps 

def show(n):
    print(n(incr)(0))

In [18]:
show(THREE)

3


In [19]:
show(FIVE)

5


In [21]:
show(TWO(THREE)) # Seems to be doing exponentiation.

9


In [ ]:
THREE(FOUR)
lambda g: THREE(FOUR(g))
lambda g: THREE(lambda x: g(g(g(g(x)))))
lambda g: THREE(FOUUR(g)(FOUR(g)(FOUR(g)(FOUR(g)(x))))

(THREE(incr)(THREE(incr)(THREE(incr)(0))))
(THREE(incr)(THREE(incr)(3)))
(THREE(incr)(6))
9

In [28]:
show(TWO(FIVE))

25


In [23]:
show(FOUR(THREE))

81


In [18]:
THREE(THREE(incr))(0)

9

In [19]:
THREE(TWO(incr))(0) # WOW!

6

### How to understand that thing?

```
let j = TWO(incr)
j(4) = 6
THREE(j)(x) = j(j(j(x)))
THREE(j)(0) = j(j(j(0))) = j(j(2)) = j(4) = 6
```

In [20]:
FOUR(THREE)(incr)(0) # Wow Wow

81

In [21]:
#Zero?

ZERO = lambda f: lambda x: x

# Notice zero has same implementation as FALSE

In [32]:
INF = lambda x: (lambda f: f(f))(lambda f: f(f)) 
# INF(42) # You can run this. Python has recursion limit.

In [33]:
ZERO(INF)(42) # Zero(f) is identity function for any f.

42

## Peano arithmetic

- 0 is a number
- if n is a number then n+1 is a number

In [34]:
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

In [35]:
SUCC(SUCC(SUCC(THREE)))(incr)(0)

6

In [36]:
incr(FOUR(incr)(0))

5

In [25]:
# ADD = lambda n: lambda m: lambda f: lambda x: m(SUCC)(n(f)(x))
ADD = lambda x: lambda y: y(SUCC)(x)

In [26]:
ADD(TWO)(THREE)(incr)(0)

5

In [60]:
MULT =  lambda x: lambda y: lambda f: x(y(f))

In [61]:
MULT(TWO)(TWO)(incr)(0)

4

# List Processing 

In [39]:
CONS = lambda a: lambda b: lambda s: s(a)(b)

In [30]:
CONS(4)(2)(LEFT)

4

In [31]:
CONS(4)(2)(RIGHT)

2

In [40]:
CAR = lambda p: p(TRUE)
CDR = lambda p: p(FALSE)

In [33]:
CAR(CONS(1)(2))

1

In [34]:
CDR(CONS(1)(2))

2

In [37]:
CAR(CONS(3)(CONS(2)(1)))

3

In [39]:
CAR(CDR(CONS(3)(CONS(2)(1))))

2

In [40]:
CDR(CDR(CONS(3)(CONS(2)(1))))

1

## How to get back to previous number?

What if you stored numbers as (n+1, n)? i.e., CONS(SUCC(n))(n)?

In [41]:
T = lambda n: CONS(SUCC(CAR(n)))(CAR(n))

In [42]:
a = FOUR(T)(CONS(ZERO)(ZERO))

In [46]:
show(CAR(a)), show(CDR(a));

4
3


In [47]:
PRED = lambda n: CDR(n(T)(CONS(ZERO)(ZERO)))

In [49]:
show(PRED(FOUR))

3


In [50]:
SUB = lambda x: lambda y: y(PRED)(x)

In [51]:
SUB(FOUR)(TWO)(incr)(0)

2

In [54]:
ISZERO = lambda n: n(lambda x: FALSE)(TRUE)

In [55]:
ISZERO(ONE)(1)(2)

2

We have build an assembly language.

AND, OR, NOT, SUCC, PRED, ADD, MUL, SUB, CONS, CAR, CDR , ISZERO

## Recursion

In [58]:
FACT = lambda n: ISZERO(n)(1)(MULT(n)(FACT(PRED(n))))

In [63]:
FACT(THREE)

RecursionError: maximum recursion depth exceeded

The reason why this didn't work is that 1,  and the other else argument (MULT(n)(FACT(PRED(n))))
are evaluated before True or False gets evaluated on these arguments.

This is not the limitation of Lambda Calculus, but of Python. Python evaluates it's arguments eagerly.

Below is a hacky patch (which introduces illegal no argument functions).

In [75]:
LAZY_TRUE = lambda x: lambda y: x()
LAZY_FALSE = lambda x: lambda y: y()
LAZY_ISZERO = lambda n: n(lambda x: LAZY_FALSE)(LAZY_TRUE)

In [78]:
FACT = lambda n: LAZY_ISZERO(n)(lambda: ONE)(lambda: MULT(n)(FACT(PRED(n))))

In [80]:
show(FACT(THREE))

6


In [81]:
show(FACT(FOUR))

24


## Annonymous functions

In lambda calculus, there is no global variables. You can't store a function into names like FACT, TRUE etc.